![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/6.Clinical_Context_Spell_Checker.ipynb)

<H1> 6. Context Spell Checker - Medical </H1>


In [ ]:
import json, os
from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp


params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.3.0
Spark NLP_JSL Version : 3.3.0


In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = RecursiveTokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")\
    .setPrefixes(["\"", "(", "[", "\n"])\
    .setSuffixes([".", ",", "?", ")","!", "'s"])

spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("token")\
    .setOutputCol("checked")

spellcheck_clinical download started this may take some time.
Approximate size to download 142.2 MB
[OK!]


In [ ]:
pipeline = Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    spellModel
  ])

empty_ds = spark.createDataFrame([[""]]).toDF("text")

lp = LightPipeline(pipeline.fit(empty_ds))

Ok!, at this point we have our spell checking pipeline as expected. Let's see what we can do with it, see these errors,

_
__Witth__ the __hell__ of __phisical__ __terapy__ the patient was __imbulated__ and on posoperative, the __impatient__ tolerating a post __curgical__ soft diet._

_With __paint__ __wel__ controlled on __orall__ pain medications, she was discharged __too__ __reihabilitation__ __facilitay__._

_She is to also call the __ofice__ if she has any __ever__ greater than 101, or __leeding__ __form__ the surgical wounds._

_Abdomen is __sort__, nontender, and __nonintended__._
            
_No __cute__ distress_

Check that some of the errors are valid English words, only by considering the context the right choice can be made.

In [ ]:
example = ["Witth the hell of phisical terapy the patient was imbulated and on posoperative, the impatient tolerating a post curgical soft diet.",
            "With paint wel controlled on orall pain medications, she was discharged too reihabilitation facilitay.",
            "She is to also call the ofice if she has any ever greater than 101, or leeding form the surgical wounds.",
            "Abdomen is sort, nontender, and nonintended.",
            "Patient not showing pain or any wealth problems.",
            "No cute distress"
            
]

for pairs in lp.annotate(example):

  print (list(zip(pairs['token'],pairs['checked'])))

[('Witth', 'With'), ('the', 'the'), ('hell', 'cell'), ('of', 'of'), ('phisical', 'physical'), ('terapy', 'therapy'), ('the', 'the'), ('patient', 'patient'), ('was', 'was'), ('imbulated', 'ambulated'), ('and', 'and'), ('on', 'on'), ('posoperative', 'postoperative'), (',', ','), ('the', 'the'), ('impatient', 'inpatient'), ('tolerating', 'tolerating'), ('a', 'a'), ('post', 'post'), ('curgical', 'surgical'), ('soft', 'soft'), ('diet', 'diet'), ('.', '.')]
[('With', 'With'), ('paint', 'paint'), ('wel', 'well'), ('controlled', 'controlled'), ('on', 'on'), ('orall', 'oral'), ('pain', 'pain'), ('medications', 'medications'), (',', ','), ('she', 'she'), ('was', 'was'), ('discharged', 'discharged'), ('too', 'too'), ('reihabilitation', 'rehabilitation'), ('facilitay', 'facility'), ('.', '.')]
[('She', 'She'), ('is', 'is'), ('to', 'to'), ('also', 'also'), ('call', 'call'), ('the', 'the'), ('ofice', 'office'), ('if', 'if'), ('she', 'she'), ('has', 'has'), ('any', 'any'), ('ever', 'ever'), ('greater